In [18]:
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate

from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain.prompts import PromptTemplate,FewShotPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [2]:
load_dotenv(dotenv_path=find_dotenv(filename="../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
examples=[
    {"input":"happy","output":"sad"},
    {"input":"tall","output":"short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [5]:
examplePrompt=PromptTemplate(
    input_variables=["input","output"],
    # The input and output are the input and output in the examples which gets mapped to the value of the input
    # So the format will be: "Input: sunny\n Output: gloomy
    template="Input: {input}\n Output:{output}",
    )

In [6]:
exampleSelector=LengthBasedExampleSelector(
    examples=examples,
    example_prompt=examplePrompt,
    max_length=15
)

In [8]:
print(exampleSelector)

examples=[{'input': 'happy', 'output': 'sad'}, {'input': 'tall', 'output': 'short'}, {'input': 'energetic', 'output': 'lethargic'}, {'input': 'sunny', 'output': 'gloomy'}, {'input': 'windy', 'output': 'calm'}] example_prompt=PromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, template='Input: {input}\n Output:{output}') get_text_length=<function _get_length_based at 0x000002297C748180> max_length=15 example_text_lengths=[4, 4, 4, 4, 4]


In [9]:
dynamicPrompt=FewShotPromptTemplate(
    prefix="Give the Antonym of every Input",
    example_selector=exampleSelector,
    example_prompt=examplePrompt,
    suffix="Input Adjective: {adjective}\nAntonym: ",
    input_variables=["adjective"]
)


In [10]:
print(dynamicPrompt.format(adjective="Untidy"))

Give the Antonym of every Input

Input: happy
 Output:sad

Input: tall
 Output:short

Input: energetic
 Output:lethargic

Input Adjective: Untidy
Antonym: 


In [15]:
chain=RunnablePassthrough()|dynamicPrompt|llmGemini|StrOutputParser()

In [16]:
print(chain.invoke(input={"adjective":"Untidy"}))

Input Adjective: Untidy
Antonym: Tidy


In [17]:
# examples -> examples_prompts -> example_selector -> few_shot_prompt_template